# Hello World: Quantum Computing with Perceval

Welcome! This notebook demonstrates how to use Perceval to simulate and run photonic quantum experiments, both locally and remotely on Quandela's Quantum Hardware. Follow along to learn the basics and try it yourself!

## 1. Install and Import Dependencies

First, let's make sure all required packages are installed and import them.  
If you haven't installed Perceval yet, run:  
`pip install perceval-quandela` in your terminal or `!pip install perceval-quandela` in your notebook.

## 2. Check Perceval Version

Let's check which version of Perceval is installed to ensure compatibility. The most up to date version is 0.13.2.

In [ ]:
import perceval as pcvl
from tqdm.notebook import tqdm
import time
pcvl.__version__

## 3. Build and Simulate a Simple Quantum Circuit

We'll create a basic quantum circuit using a beam splitter and simulate it locally.  
This helps us understand the basics before moving to the cloud.

![HOM](./img/HOM_effect.png)

In [ ]:
from perceval.algorithm import Sampler

input_state = pcvl.BasicState("|1,1>")  # Inject one photon on each input mode...
circuit = pcvl.BS()                     # ... of a perfect beam splitter
noise_model = pcvl.NoiseModel(transmittance=0.05, indistinguishability=0.85, g2=0.03)  # Define some noise level (No noise if: transmittance=1, indistinguishability=1, g2=0)

processor = pcvl.Processor("SLOS", circuit, noise=noise_model)  # Use SLOS, a strong simulation back-end
processor.min_detected_photons_filter(1)  # Accept all output states containing at least 1 photon
processor.with_input(input_state)

nsamples = 200_000  # Number of samples to generate

sampler = Sampler(processor)
samples = sampler.sample_count(nsamples)['results']  # Ask to generate 10k samples, and get back only the raw results
probs = sampler.probs()['results']  # Ask for the exact probabilities
print(f"Samples: {samples}")
print(f"Probabilities: {probs}")

## 4. Configure Cloud Access

To run jobs on Quandela's real quantum hardware, you'll need an API token.  
Follow the instructions to set up your credentials securely.

In [ ]:
from perceval import RemoteConfig, RemoteProcessor
# Save your token and proxy configuration into Perceval persistent data, you only need to do it once per machine.
remote_config = RemoteConfig()
remote_config.set_token("ENTER_YOUR_TOKEN_HERE")  # Replace with your Quandela token which you can create in you Quandela account at https://www.cloud.quandela.com/
remote_config.save()

## 5. Run a Job on Quandela's Quantum Computer

Now, let's submit our circuit to the cloud and monitor the job's progress.
If you change the platform from sim:slos to qpu:ascella the job will be send not to the simulator anymore but to the quantum computer.

In [ ]:
remote_processor = RemoteProcessor("qpu:ascella", noise=noise_model)
remote_processor.set_circuit(circuit)
remote_processor.min_detected_photons_filter(1)
remote_processor.with_input(input_state)

remote_sampler = Sampler(remote_processor, max_shots_per_call=1e6)
remote_sampler.default_job_name = "Hello World" 
remote_job = remote_sampler.sample_count.execute_async(nsamples)
print(remote_job.id)

## 6. Monitor Job Progress

We'll use a progress bar to track the status of our cloud job in real time.

Or you can go to your Quandela Account and see the Job you have just created there. (Note you can see in the Platform column if the job was run on a simulator or on teh quantum computer)

![Cloud Job Status](./img/FirstCloudJob.png)

In [ ]:
previous_prog = 0
with tqdm(total=1, bar_format='{desc}{percentage:3.0f}%|{bar}|') as tq:
    tq.set_description(f'Get {nsamples} samples from {remote_processor.name}')
    while not remote_job.is_complete:
        tq.update(remote_job.status.progress/100-previous_prog)
        previous_prog = remote_job.status.progress/100
        time.sleep(1)
    tq.update(1-previous_prog)
    tq.close()

print(f"Job status = {remote_job.status()}")

## 7. Retrieve and Display Results

Once the job is complete, let's fetch and visualize the results from the quantum hardware. 

This can be done directly:

In [ ]:
results = remote_job.get_results()
pcvl.pdisplay(results['results'])

Or later with the job ID:

In [ ]:
job_id = "put_your_job_id_here"  # Replace with your job id
remote_processor = RemoteProcessor("put_your_platform_name_here")
remote_job = remote_processor.resume_job(job_id) # even though it says resume job this also works for fetching an already completed job

# The remote job acts as if it had been created the first time: you can launch it, get the results...
results = remote_job.get_results()   # If the job has already been run and results are available 
pcvl.pdisplay(results['results'])

## 8. Extra Tools

You can visualize your circuit with Perceval:

In [ ]:
pcvl.pdisplay(circuit)

You can get the current performance parameters of our QPUs:

The optimal performance is: HOM = 100%, Transmittance = 100% and g2 = 0%.

The Clock tells you how many photons are generated per second. (80 MHz means 80 million photons per second)

In [ ]:
ascella = pcvl.RemoteProcessor("qpu:ascella")
perf_ascella = ascella.performance
print(f"The Performance of Ascella is: {perf_ascella}")

belenos = pcvl.RemoteProcessor("qpu:belenos")
perf_belenos = belenos.performance
print(f"The Performance of Belenos is: {perf_belenos}")

# Conclusion

Congratulations! You've simulated and run a quantum photonic circuit on real hardware using Perceval and Quandela's cloud platform.  
Feel free to experiment with different circuits and parameters!